# Step 2 - Generate Candidates

Our goal during this step is to generate candidate mids.

In [1]:
import sys
sys.path.insert(0, '../../')

In [2]:
from lib.connect import get_connection 
from lib.data import FB2M_NAME_TABLE

connection = get_connection()
cursor = connection.cursor()

In [33]:
import pandas as pd
from tqdm import tqdm_notebook

tqdm_notebook().pandas()

df = pd.read_pickle('step_1_predict_subject_name.pkl')
df[:5]

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

,end_index,object,predicted_question_tokens,predicted_subject_names,question,question_tokens,relation,start_index,subject,subject_name,subject_name_tokens
17188,7.0,075s73,"[which, town, is, in, new, york, city]","[{'name': 'new york city', 'score': 64.1403045...",which town is in new york city,"[which, town, is, in, new, york, city]",location/place_with_neighborhoods/neighborhoods,4.0,02_286,new york city,"(new, york, city)"
4793,4.0,017drs,"[does, pee, wee, reese, play, shortstop, or, p...","[{'name': 'pee wee reese', 'score': 91.0824661...",does pee wee reese play shortstop or power for...,"[does, pee, wee, reese, play, shortstop, or, p...",baseball/baseball_player/position_s,1.0,01bg1k,pee wee reese,"(pee, wee, reese)"
21187,7.0,0vp3fq,"[what, is, a, track, by, lutricia, mcneal, ?]","[{'name': 'lutricia mcneal', 'score': 79.92938...",What is a track by lutricia mcneal?,"[what, is, a, track, by, lutricia, mcneal, ?]",music/artist/track,5.0,06tw28,lutricia mcneal,"(lutricia, mcneal)"
18730,6.0,0wzyx1,"[name, a, recording, by, nelson, mandela]","[{'name': 'nelson mandela', 'score': 57.336029...",Name a recording by nelson mandela,"[name, a, recording, by, nelson, mandela]",music/release_track/recording,4.0,0slws_1,nelson mandela,"(nelson, mandela)"
10014,9.0,0crryw4,"[name, a, film, in, the, netflix, genre, celti...","[{'name': 'celtic music', 'score': 106.7388153...",Name a film in the netflix genre celtic music.,"[name, a, film, in, the, netflix, genre, celti...",media_common/netflix_genre/titles,7.0,01m1y,celtic music,"(celtic, music)"


Define text preprocessing the same as the training data and step 1.

In [4]:
import importlib
import lib.import_notebook
import re
from nltk.stem.snowball import SnowballStemmer

STEMMER = SnowballStemmer("english")
PREPROCESS = importlib.import_module(
                "notebooksSimple QA Models.Subject Recognition Data").preprocess
TOKENIZE = importlib.import_module(
                "notebooksSimple QA Models.Subject Recognition Data").spacy_tokenize

def text_preprocess(s):
    # Define `text_preprocess` the way the input text was preprocessed before step 1
    s = PREPROCESS(s)
    s = TOKENIZE(s)
    s = ' '.join(s)
    return s

def text_normalize_punctuation(s):
    s = text_preprocess(s)
    # In `Normalized Reference Resolution#HYPOTHESIS - Subject Name not in Question.ipynb` we found that
    # aliases and questions match up more if punctuation is removed.
    # Remove punctuation
    s = re.sub(r'[^\w\s]','',s)
    # Removing characters can create gaps of multiple spaces
    # Substitue multiple spaces with one
    s = re.sub('\s+', ' ', s)
    s = s.strip()
    return s


def text_normalize_punctuation_stem(s):
    s = text_preprocess(s)
    
    # Remove Possessives
    tokens = s.split()
    possessives = ["'s"]
    tokens = [t for t in tokens if t not in possessives]
    # Stem
    tokens = [STEMMER.stem(t) for t in tokens]
    s = ' '.join(tokens)
    
    s = text_normalize_punctuation(s)
    return s

importing Jupyter notebook from ../../notebooks/Simple QA Models/Subject Recognition Data.ipynb


## Index Subject Aliases

Create an index of subject aliases that are preprocessed similar to the predicted subect name. Allowing for a database lookup.

In [ ]:
cursor.execute('ALTER TABLE ' + FB2M_NAME_TABLE + ' ADD COLUMN alias_normalized_punctuation varchar')

In [ ]:
cursor.execute('ALTER TABLE ' + FB2M_NAME_TABLE + ' ADD COLUMN alias_normalized_punctuation_stem varchar')

In [ ]:
cursor.execute('ALTER TABLE ' + FB2M_NAME_TABLE + ' ADD COLUMN alias_preprocessed varchar')

In [ ]:
from tqdm import tqdm_notebook
import psycopg2

chunk_size = 10000

def update_chunk(rows):
    query = ('UPDATE ' + FB2M_NAME_TABLE + ' SET alias_preprocessed = %s, ' +
             'alias_normalized_punctuation = %s, alias_normalized_punctuation_stem = %s ' +
             'WHERE mid = %s and alias = %s')
    psycopg2.extras.execute_batch(cursor, query, rows)

cursor.execute('SELECT mid, alias FROM ' + FB2M_NAME_TABLE)
rows = []
for mid, alias in tqdm_notebook(cursor.fetchall()):
    alias_preprocessed = text_preprocess(alias)
    alias_normalized_punctuation = text_normalize_punctuation(alias)
    alias_normalized_punctuation_stem = text_normalize_punctuation_stem(alias)
    rows.append(tuple([alias_preprocessed, alias_normalized_punctuation, 
                       alias_normalized_punctuation_stem, mid, alias]))
    
    # Insert Chunk
    if len(rows) > chunk_size:
        update_chunk(rows)
        rows = []
        
update_chunk(rows)

In [ ]:
connection.commit()

In [ ]:
cursor.execute('CREATE INDEX ' + FB2M_NAME_TABLE + '_alias_preprocessed ON ' + 
               FB2M_NAME_TABLE + '(alias_preprocessed);')
connection.commit()

In [ ]:
cursor.execute('CREATE INDEX ' + FB2M_NAME_TABLE + '_alias_normalized_punctuation ON ' + 
               FB2M_NAME_TABLE + '(alias_normalized_punctuation);')
connection.commit()

In [ ]:
cursor.execute('CREATE INDEX ' + FB2M_NAME_TABLE + '_alias_normalized_punctuation_stem ON ' + 
               FB2M_NAME_TABLE + '(alias_normalized_punctuation_stem);')
connection.commit()

In [ ]:
cursor.execute('CREATE INDEX ' + FB2M_NAME_TABLE + '_alias_normalized_punctuation_stem_trgm ON ' + 
               FB2M_NAME_TABLE + ' USING gist(alias_normalized_punctuation_stem gist_trgm_ops);')
connection.commit()

## Generate Candidates

If subject name is null, then the question does not refer to the true alias. The example is then unanswerable.

In [5]:
df_answerable = df[df.subject_name.notnull()]

Metrics used to evaluate different versions.

In [6]:
def evaluate_candidates(candidates_mids):
    correct = 0
    skipped = 0
    expected_accuracy = 0
    n_answerable_examples = df_answerable.shape[0]
    n_examples = df.shape[0]

    for i, (_, row) in enumerate(df_answerable.iterrows()):
        mids = candidates_mids[i]
        if len(mids) == 0:
            skipped += 1
        elif row['subject'] in mids:
            correct += 1
            expected_accuracy += 1 / len(mids)
        
    print('Answerable Precision: %f [%d of %d]' %
              (correct / (n_answerable_examples - skipped), correct,
               (n_answerable_examples - skipped)))
    print('Answerable Recall: %f [%d of %d]' %
              ((n_answerable_examples - skipped) / n_answerable_examples,
               (n_answerable_examples - skipped), n_answerable_examples))
    print('Expected Guessing Accuracy: %f [%d of %d]' % 
              (expected_accuracy / n_examples, expected_accuracy, n_examples))

Basic helper functions to run experiments quickly.

In [8]:
from functools import lru_cache

@lru_cache(maxsize=65536)
def cached_alias_to_mid(text):
    cursor.execute("SELECT mid FROM " + FB2M_NAME_TABLE +  
                  " WHERE alias = %s", (text,))
    return list([r[0] for r in cursor.fetchall()])

def cached_aliases_to_mids(aliases):
    mids = []
    for alias in aliases:
        mids.extend(cached_alias_to_mid(alias))
    return mids

@lru_cache(maxsize=65536)
def cached_alias_normalized_punctuation_to_alias(text):
    cursor.execute("SELECT DISTINCT alias FROM " + FB2M_NAME_TABLE + 
                  " WHERE alias_normalized_punctuation = %s", (text,))
    return list([r[0] for r in cursor.fetchall()])

@lru_cache(maxsize=65536)
def cached_alias_preprocessed_to_alias(text):
    cursor.execute("SELECT DISTINCT alias FROM " + FB2M_NAME_TABLE + 
                  " WHERE alias_preprocessed = %s", (text,))
    return list([r[0] for r in cursor.fetchall()])


@lru_cache(maxsize=65536)
def cached_alias_normalized_punctuation_stem_to_alias(text):
    cursor.execute("SELECT DISTINCT alias FROM " + FB2M_NAME_TABLE + 
                  " WHERE alias_normalized_punctuation_stem = %s", (text,))
    return list([r[0] for r in cursor.fetchall()])

## Generate Candidates - Upperbound

Here we use the true alias, to compute the upperbound of this task.

In [9]:
candidates_mids = []

for index, row in tqdm_notebook(df_answerable.iterrows(), total=df_answerable.shape[0]):
    candidate_aliases = [row['subject_name']]
    candidates_mids.append(cached_aliases_to_mids(candidate_aliases))        

evaluate_candidates(candidates_mids)

HBox(children=(IntProgress(value=0, max=21188), HTML(value='')))


Answerable Precision: 1.000000 [21188 of 21188]
Answerable Recall: 1.000000 [21188 of 21188]
Expected Guessing Accuracy: 0.667733 [14427 of 21607]


## Generate Candidates - Baseline

Just lookup the top k predicted subject names in order until one is seen.

In [10]:
from lib.table import format_pipe_table

negative_sample = []
candidates_mids = []

for index, row in tqdm_notebook(df_answerable.iterrows(), total=df_answerable.shape[0]):
    for predicted in row['predicted_subject_names']:
        candidate_aliases = cached_alias_preprocessed_to_alias(predicted['name'])
    
        if len(candidate_aliases) > 0:
            candidates_mids.append(cached_aliases_to_mids(candidate_aliases))
            break
            
    if len(candidate_aliases) == 0:
        candidates_mids.append([])
        

evaluate_candidates(candidates_mids)

HBox(children=(IntProgress(value=0, max=21188), HTML(value='')))


Answerable Precision: 0.963693 [20332 of 21098]
Answerable Recall: 0.995752 [21098 of 21188]
Expected Guessing Accuracy: 0.634341 [13706 of 21607]


### Version 1

For the first version, we will try to follow the strategy in `Normalized Reference Resolution#HYPOTHESIS - Subject Name not in Question.ipynb` to link more aliases to questions.

In [11]:
# Helper method to play with the metric
def pg_trgm_similarity(text, other_text):
    cursor.execute('SELECT similarity(%s, %s);', (text, other_text))
    similarity = cursor.fetchall()[0][0]
    return similarity
                   
# TEST
print(pg_trgm_similarity('hi', 'hey'))

0.166667


In [12]:
from lib.table import format_pipe_table

negative_samples = []
candidates_mids = []

for index, row in tqdm_notebook(df_answerable.iterrows(), total=df_answerable.shape[0]):
    for i, predicted in enumerate(row['predicted_subject_names']):
        strategy = 'PREPROCESSED'
        candidate_aliases = cached_alias_preprocessed_to_alias(predicted['name'])
        
        # Punctuation Differences
        if len(candidate_aliases) == 0:
            # NOTE: Normalized alias has a broader reach; therefore, we only use it if the first check failed.
            # We found this increased precision and expected guessing accuracy to add the check.
            strategy = 'NORMALIZED_PUNCTUATION'
            candidate_aliases = cached_alias_normalized_punctuation_to_alias(
                text_normalize_punctuation(predicted['name']))
    
        if len(candidate_aliases) > 0:
            candidates_mids.append(cached_aliases_to_mids(candidate_aliases))
            if row['subject'] not in candidates_mids[-1]:
                considered_aliases = [predicted['name'] for j, predicted in 
                                          enumerate(row['predicted_subject_names']) if j <= i]
                negative_samples.append({
                    'Preprocessed Subject Name': text_preprocess(row['subject_name']),
                    'Considered Aliases': considered_aliases,
                    'Max Similarity': max([pg_trgm_similarity(row['subject_name'], a)
                                           for a in considered_aliases]),
                    'Predicted Alias': predicted['name'],
                    'Strategy': strategy,
                    'Question': row['question'],
                })
            break
            
    if len(candidate_aliases) == 0:
        candidates_mids.append([])

evaluate_candidates(candidates_mids)
print('Negative Sample:')
print(format_pipe_table(negative_samples[:50], columns=['Strategy', 'Max Similarity',
                                                        'Preprocessed Subject Name',
                                                        'Predicted Alias',
                                                        'Considered Aliases', 'Question']))

HBox(children=(IntProgress(value=0, max=21188), HTML(value='')))

importing Jupyter notebook from ../../notebooks/Simple QA Numbers/HYPOTHESIS - Subject Name not in Question.ipynb

Answerable Precision: 0.967804 [20471 of 21152]
Answerable Recall: 0.998301 [21152 of 21188]
Expected Guessing Accuracy: 0.639845 [13825 of 21607]
Negative Sample:
| Index | Strategy | Max Similarity | Preprocessed Subject Name | Predicted Alias | Considered Aliases | Question |
| --- | --- | --- | --- | --- | --- | --- |
| 0 | PREPROCESSED | 0.0 | ken hirai | album | ['album'] | what is the cover album by Japanese pop singer Ken Hirai |
| 1 | PREPROCESSED | 0.714286 | st helens rfc | st helens | ['st helens rlfc', 'st helens'] | What is a color of the st helens rlfc |
| 2 | PREPROCESSED | 0.0 | thriller | felix enriquez alcala | ['felix enriquez alcala'] | what thriller was directed by Félix Enríquez Alcalá |
| 3 | PREPROCESSED | 1.0 | who 's that girl ? | who s that girl | ['who s that girl'] | what track came off  whos that girl? |
| 4 | PREPROCESSED | 0.0 | black - and

#### Analysis

##### Numbers:

Version 0
- Precision: 0.964420 [10246 of 10624]
- Recall: 0.997746 [10624 of 10648]
- Expected Guessing Accuracy: 0.659801 [7025 of 10648]

Version 1
- Precision: 0.968524 [10308 of 10643]
- Recall: 0.999530 [10643 of 10648]
- Expected Guessing Accuracy: 0.664496 [7075 of 10648]

Recall increased by 0.001784.
Precision increased by 0.004104.


##### Error Bucket:

**Discussion:**

Handling possesives would fix 10 / 50 errors. Handling the `Similar` bucket would be difficult because it's typically because of extra words in the subject name not present in the question.

**Buckets:**
- Wrong Span (29 / 50): The wrong span in the question was selected
- Suffix (12 / 50): The correct subject name was not linked due to a suffix.
- Extra Article (3 / 50): The correct subject name was not linked due to an article.
- Similar (7 / 50): The correct subject name was similar but not exact to the predicted subject name.
- Other (1 / 50): Deeper reason that the correct subject name was not linked.

| Index | Similarity | Bucket | Strategy | Preprocessed Subject Name | Predicted Alias | Considered Aliases | Question |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 0 | 0.0 | Wrong Span | PREPROCESSED | short | documentary film | ['documentary film'] | Name a short documentary film released in 2011 |
| 1 | 0.722222 | Suffix | PREPROCESSED | red cloud 's war | the red | ['the red clouds war', 'red clouds war', 'the red clouds', 'clouds war', 'red clouds', 'the red'] | what was involved in the red clouds war? |
| 2 | 0.75 | Wrong Span | PREPROCESSED | corporation nation | nation | ['nation book', 'corporation nation book', 'the corporation nation book', 'nation'] | what subject is the corporation nation book about |
| 3 | 0.8 | Suffix | PREPROCESSED | peter 's point plantation | peters | ['peters point plantation', 'peters point', 'point plantation', 'peters'] | What is peters point plantation's architectural style |
| 4 | 0.0555556 | Wrong Span | PREPROCESSED | album | aaron carter | ['aaron carter'] | Name an album released by aaron carter |
| 5 | 1.0 | Similar | PREPROCESSED | pillows & prayers : cherry red 1982–1983 | pillows & prayers : cherry red 1982 - 1983 | ['pillows & prayers : cherry red 1982 - 1983'] | What is the name of the track list for the release pillows & prayers: cherry red 1982-1983? |
| 6 | 0.5 | Wrong Span | PREPROCESSED | commune of luxembourg | luxembourg | ['luxembourg'] | which country is the commune of luxembourg in |
| 7 | 0.764706 | Extra Article | PREPROCESSED | the hits album 6 | 6 | ['hits album 6', '6'] | what song was included in the hits album 6 |
| 8 | 0.588235 | Wrong Span | PREPROCESSED | between two women | two women | ['two women'] | what is about between two women |
| 9 | 0.782609 | Suffix | PREPROCESSED | battle of hudson 's bay | bay | ['battle of hudsons bay', 'of hudsons bay', 'battle of hudsons', 'the battle of hudsons bay', 'hudsons bay', 'did the battle of hudsons bay', 'battle of', 'bay'] | where did the battle of hudsons bay take place |
| 10 | 0.0 | Wrong Span | PREPROCESSED | tablet | hypertension | ['hypertension'] | what is a tablet used to treat hypertension  |
| 11 | 0.0 | Wrong Span | PREPROCESSED | compilation album | frank zappa | ['frank zappa'] | what compilation album did frank zappa release? |
| 12 | 0.0 | Wrong Span | PREPROCESSED | soundtrack | anthony marinelli | ['anthony marinelli'] | What's a soundtrack written by anthony marinelli |
| 13 | 0.0 | Wrong Span | PREPROCESSED | album | george canyon | ['george canyon'] | name an album by George Canyon |
| 14 | 0.0 | Wrong Span | PREPROCESSED | album | portal | ['portal'] | What's an album by the band portal |
| 15 | 0.785714 | Suffix | PREPROCESSED | megan cheng | megan | ['megan chengs', 'megan'] | whats  megan chengs ethnicity |
| 16 | 0.705882 | Wrong Span | PREPROCESSED | martial arts film | martial arts | ['martial arts'] | what is the name of the netflix martial arts film? |
| 17 | 0.0222222 | Wrong Span | PREPROCESSED | creedence clearwater revival | compilation album | ['compilation album'] | What is a compilation album by creedence clearwater revival |
| 18 | 0.227273 | Wrong Span | PREPROCESSED | topical medication | medicine | ['medicine'] | Name a topical medicine |
| 19 | 0.636364 | Wrong Span | PREPROCESSED | master | the master | ['the master'] | what is one of the master's powers  |
| 20 | 0.0 | Other | PREPROCESSED | t - town | kearny | ['kearny'] | What newspaper circulates in the town of kearny |
| 21 | 0.571429 | Suffix | PREPROCESSED | drums | drum | ['drum'] | which musician plays the drum kit |
| 22 | 0.84375 | Suffix | PREPROCESSED | dimillo 's floating restaurant | restaurant | ['dimillos floating restaurant', 'dimillos floating', 'floating restaurant', 'dimillos', 'is dimillos floating restaurant', 'dimillos floating restaurant in', 'restaurant'] | what state is dimillos floating restaurant in? |
| 23 | 0.0 | Wrong Span | PREPROCESSED | ragtime | denmark | ['denmark'] | who is the ragtime artist born in denmark? |
| 24 | 0.8 | Similar, Extra Article | PREPROCESSED | the regatta mystery | mystery | ['regatta mystery', 'mystery'] | what theme is in the piece regatta mystery |
| 25 | 0.0 | Wrong Span | PREPROCESSED | album | jack | ['jack dejohnrette', 'jack'] | What is the name of Jack DeJohnrette's album? |
| 26 | 0.0 | Wrong Span | PREPROCESSED | bollywood | tamil | ['tamil'] | what bollywood Tamil film was released in 2004  |
| 27 | 0.0 | Wrong Span | PREPROCESSED | animated cartoon | ducks | ['ducks'] | what animated cartoon was about ducks? |
| 28 | 0.0 | Wrong Span | PREPROCESSED | photography | visual art | ['visual art'] | which artist uses photography as their preferred visual art form |
| 29 | 0.761905 | Suffix | PREPROCESSED | this pud 's for you | for you | ['this puds for you comes', 'this puds for you', 'this puds for you comes from', 'puds for you comes', 'puds for you', 'this puds for', 'this puds', 'episode this puds for you comes', 'for you comes', 'puds for you comes from', 'episode this puds for you', 'for you'] | what is the series where the episode this puds for you comes from |
| 30 | 0.826087 | Suffix | NORMALIZED | chet 's speech , part ii | , part ii | ['chets speech , part ii', 'speech , part ii', 'chets speech , part', 'chets speech ,', 'chets speech', ', part ii'] | who sings chets speech, part ii |
| 31 | 0.764706 | Wrong Span | PREPROCESSED | large family car | family | ['large family', 'family'] | What car model is an example of a large family car? |
| 32 | 0.761905 | Suffix | PREPROCESSED | men 's pommel horse | pommel horse | ['mens pommel horse', 'mens pommel', 'pommel horse'] | What olympic games featured mens pommel horse |
| 33 | 0.0526316 | Wrong Span | NORMALIZED | soundtrack | s.cry.ed | ['s.cry.ed'] | What's the soundtrack for s.cry.ed |
| 34 | 0.35 | Wrong Span | PREPROCESSED | sahara ( instrumental ) | sahara | ['sahara'] | who composed sahara (instrumental)? |
| 35 | 0.0625 | Wrong Span | PREPROCESSED | compilation | cema | ['albumby cema', 'cema'] | what album is released as a compilation albumby CEMA |
| 36 | 0.583333 | Wrong Span | PREPROCESSED | arabic name | arabic | ['arabic'] | What is a book that is about arabic name |
| 37 | 0.73913 | Similar | PREPROCESSED | multiplayer video game | game | ['multiplayer game', 'game'] | What's a text based multiplayer game |
| 38 | 0.75 | Extra Article | PREPROCESSED | the crystal city | crystal city | ['crystal city'] | what genre is crystal city |
| 39 | 0.84 | Suffix | PREPROCESSED | men 's badminton , singles | singles | ['mens badminton , singles', 'badminton , singles', 'mens badminton', 'mens badminton ,', 'singles'] | what olympic games was mens badminton, singles apart of |
| 40 | 0.0 | Wrong Span | PREPROCESSED | mercedes lackey | fantasy | ['fantasy'] | which fantasy series were written by mercedes lackey? |
| 41 | 0.666667 | Similar | PREPROCESSED | brian o'shea | brian oshea | ['brian oshea'] | brian oshea performs what type of martial art |
| 42 | 0.857143 | Similar | PREPROCESSED | u.s . office of war information | war | ['office of war information', 'office of war information help', 'the office of war information', 'war information', 'the office of war information help', 'of war information', 'office of war', 'war information help', 'of war information help', 'the office of war', 'office of war information help produce', 'war'] | which film did the office of war information help produce  |
| 43 | 0.0 | Wrong Span | PREPROCESSED | album | sham 69 | ['sham 69'] | which album is released by Sham 69 |
| 44 | 0.777778 | Wrong Span | NORMALIZED | lowthian bell | , 1st baronet | ['sir lowthian bell , 1st baronet', 'lowthian bell , 1st baronet', 'sir lowthian bell , 1st', 'sir lowthian bell ,', 'sir lowthian bell', 'bell , 1st baronet', 'sir lowthian', ', 1st baronet'] | what organization was founded by sir lowthian bell, 1st baronet |
| 45 | 0.862069 | Suffix | PREPROCESSED | st . peter 's episcopal church | st . peters | ['st . peters episcopal church', 'st . peters episcopal', '. peters episcopal church', 'peters episcopal church', 'st . peters'] | what state and city is st. peters episcopal church located in? |
| 46 | 0.851852 | Suffix | PREPROCESSED | richard scarry 's busytown | busytown | ['richard scarrys busytown', 'scarrys busytown', 'richard scarrys', 'busytown'] | what is a gameplay mode featured on richard scarrys busytown |
| 47 | 0.0 | Wrong Span | PREPROCESSED | album | soil | ['soil'] | What's an album by soil |
| 48 | 0.714286 | Similar | PREPROCESSED | texas a&m university school of law | texas wesleyan university | ['texas wesleyan university school of law', 'wesleyan university school of law', 'texas wesleyan university school of', 'texas wesleyan university school', 'university school of law', 'is texas wesleyan university school of law', 'texas wesleyan university'] | Where is texas wesleyan university school of law located? |
| 49 | 0.535714 | Wrong Span | PREPROCESSED | public service announcement | public service | ['public service'] | What is the name of a public service announcement? |




## Version 2

In version 1, the error bucketing revealed a failure to handle suffix's; therefore, we proceed to handle them in version 2.

In [13]:
from lib.table import format_pipe_table

negative_samples = []
candidates_mids = []

for index, row in tqdm_notebook(df_answerable.iterrows(), total=df_answerable.shape[0]):
    for i, predicted in enumerate(row['predicted_subject_names']):
        strategy = 'PREPROCESSED'
        candidate_aliases = cached_alias_preprocessed_to_alias(predicted['name'])
        
        # Punctuation Differences
        if len(candidate_aliases) == 0:
            # NOTE: Normalized alias has a broader reach; therefore, we only use it if the first check failed.
            # We found this increased precision and expected guessing accuracy to add the check.
            strategy = 'NORMALIZED_PUNCTUATION'
            candidate_aliases = cached_alias_normalized_punctuation_to_alias(
                text_normalize_punctuation(predicted['name']))
            
        # Suffix Differences
        if len(candidate_aliases) == 0:
            # NOTE: Normalized alias has a broader reach; therefore, we only use it if the first check failed.
            # We found this increased precision and expected guessing accuracy to add the check.
            strategy = 'NORMALIZED_PUNCTUATION_STEM'
            candidate_aliases = cached_alias_normalized_punctuation_stem_to_alias(
                text_normalize_punctuation_stem(predicted['name']))
    
        if len(candidate_aliases) > 0:
            candidates_mids.append(cached_aliases_to_mids(candidate_aliases))
            if row['subject'] not in candidates_mids[-1]:
                considered_aliases = [predicted['name'] for j, predicted in 
                                          enumerate(row['predicted_subject_names']) if j <= i]
                negative_samples.append({
                    'Preprocessed Subject Name': text_preprocess(row['subject_name']),
                    'Considered Aliases': considered_aliases,
                    'Max Similarity': max([pg_trgm_similarity(text_normalize_punctuation_stem(row['subject_name']),
                                                              text_normalize_punctuation_stem(a))
                                           for a in considered_aliases]),
                    'Predicted Alias': predicted['name'],
                    'Strategy': strategy,
                    'Question': row['question'],
                })
            break
            
    if len(candidate_aliases) == 0:
        candidates_mids.append([])

evaluate_candidates(candidates_mids)
print('Negative Sample:')
# To not overfit on the first 50 samples
print(format_pipe_table(negative_samples[50:100], columns=['Strategy', 'Max Similarity',
                                                        'Preprocessed Subject Name',
                                                        'Predicted Alias',
                                                        'Considered Aliases', 'Question']))

HBox(children=(IntProgress(value=0, max=21188), HTML(value='')))


Answerable Precision: 0.972742 [20591 of 21168]
Answerable Recall: 0.999056 [21168 of 21188]
Expected Guessing Accuracy: 0.644360 [13922 of 21607]
Negative Sample:
| Index | Strategy | Max Similarity | Preprocessed Subject Name | Predicted Alias | Considered Aliases | Question |
| --- | --- | --- | --- | --- | --- | --- |
| 0 | PREPROCESSED | 0.0 | armenians | novelist | ['novelist'] | who is a famous armenians novelist and playwright  |
| 1 | PREPROCESSED | 0.238095 | jojo 's bizarre adventure | jojo | ['jojo'] | Who is a character of jojo's bizarre adventure? |
| 2 | PREPROCESSED | 0.0 | seychelles | praslin | ['praslin'] | What is a administrative district in seychelles that is located on the island of Praslin? |
| 3 | PREPROCESSED | 0.0 | album | elmore james | ['the history of elmore james', 'history of elmore james', 'of elmore james', 'elmore james'] | what is an album title including the history of elmore james |
| 4 | PREPROCESSED | 1.0 | crime dramas | crime drama | ['crime 

#### Analysis

##### Numbers:

Version 0
- Precision: 0.964420 [10246 of 10624]
- Recall: 0.997746 [10624 of 10648]
- Expected Guessing Accuracy: 0.659801 [7025 of 10648]

Version 1
- Precision: 0.968524 [10308 of 10643]
- Recall: 0.999530 [10643 of 10648]
- Expected Guessing Accuracy: 0.664496 [7075 of 10648]
    
Verison 2
- Precision: 0.973420 [10364 of 10647]
- Recall: 0.999906 [10647 of 10648]
- Expected Guessing Accuracy: 0.669337 [7127 of 10648]

Recall increased by 0.000376.

Precision increased by 0.004896.

**Discussion:**

We managed to increase our expected accuracy by a 1% with these simple normalization measures. We believe this will lead to a 1% + gain downstream.

##### Error Bucket:

**Discussion:**

For the most part, the wrong span is selected; therefore, the alias is difficult to link with the current implementation of the algorithm.

We can handle the similar case some what. The similarity between the correct alias and the subject name tends to be fairly high; therefore, we can try adding a step to look for any aliases that are similar with a score of 0.85+. We expect this to handle  3 / 50 errors.

| Max Similarity | Bucket |
| --- | --- |
| 0.888889 | Similar |
| 0.714286 | Similar |
| 0.875 | Similar |
| 1.0 | Similar |
| 0.62069 | Similar |
| 1.0 | Similar |
| 0.88 | Similar |

**Buckets:**
- Wrong Span (42 / 50): The wrong span in the question was selected.
- Similar (8 / 50): The correct subject name was similar but not exact to the predicted subject name.
- Extra Article (2 / 50): The correct subject name was not linked due to an article.

| Index | Max Similarity  | Bucket | Strategy | Preprocessed Subject Name | Predicted Alias | Considered Aliases | Question |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 0 | 0.705882 | Wrong Span | PREPROCESSED | pim fortuyn list | pim fortuyn | ['pim fortuyn'] | what ideology does the pim fortuyn list follow |
| 1 | 0.0 | Wrong Span | NORMALIZED_PUNCTUATION_STEM | drug | cleaning | ['cleaning hands', 'for cleaning hands', 'cleaning'] | what drug is used for cleaning hands  |
| 2 | 0.444444 | Wrong Span | PREPROCESSED | leather subculture | leather | ['leather'] | what's one event that celebrates the leather subculture |
| 3 | 0.333333 | Wrong Span | PREPROCESSED | illinois river | rogue river | ['lower rogue river', 'rogue river'] | in which community does the illinois river confluence with the Lower Rogue river |
| 4 | 0.764706 | Wrong Span, Extra Article | PREPROCESSED | need for speed | the need for speed | ['the need for speed'] | what type of film is the need for speed |
| 5 | 0.0 | Wrong Span | PREPROCESSED | asteroid | geologist | ['geologist'] | which asteroid was names after an italian geologist? |
| 6 | 0.0 | Wrong Span | PREPROCESSED | album | chico debarge | ['chico debarge'] | what is an album by Chico DeBarge? |
| 7 | 1.0 | Punctuation | PREPROCESSED | unter null . | unter null | ['unter null'] | what type of book binding is unter null. |
| 8 | 0.5 | Wrong Span | NORMALIZED_PUNCTUATION | k - pop | - pop music | ['k - pop music', '- pop music'] | Who is an artist of k-pop music? |
| 9 | 0.0454545 | Wrong Span | PREPROCESSED | novel | vladimir nabokov | ['vladimir nabokov'] | Name a novel by Vladimir Nabokov |
| 10 | 0.888889 | Similar | PREPROCESSED | millard s drexler | millard | ['millard drexler', 'drexler', 'millard'] | What organization did millard drexler found |
| 11 | 0.0 | Wrong Span | PREPROCESSED | musical | israel | ['israel'] | which musical films were broadcasted in israel? |
| 12 | 0.0 | Wrong Span | PREPROCESSED | live album | 3oh!3 | ['3oh!3'] | what is the name of the live album by 3oh!3 |
| 13 | 0.5 | Wrong Span | PREPROCESSED | epic film | epic | ['epic'] | Name a 1936 epic film  |
| 14 | 0.533333 | Wrong Span | PREPROCESSED | roy rogers restaurants | roy rogers | ['roy rogers'] | roy rogers restaurants in which industry? |
| 15 | 0.0 | Wrong Span | PREPROCESSED | caucasian | babylon 5 | ['babylon 5'] | who is of caucasian race in babylon 5 |
| 16 | 0.6 | Wrong Span | PREPROCESSED | painting | visual art | ['visual art painting', 'art painting', 'visual art'] | what is a visual art painting |
| 17 | 0.0 | Wrong Span | PREPROCESSED | album | funk | ['koul funk', 'funk'] | what album was release by Koul Funk |
| 18 | 0.0555556 | Wrong Span | PREPROCESSED | science | brian swimme | ['brian swimme'] | brian swimme wrote what book that dealt with  science |
| 19 | 0.0 | Wrong Span | PREPROCESSED | animation | raoul servais | ['raoul servais'] | Which animation did Raoul Servais direct |
| 20 | 0.333333 | Wrong Span | PREPROCESSED | working title films | films | ['films'] | what is the film from the production company working title films |
| 21 | 0.0344828 | Wrong Span | PREPROCESSED | jerry bruckheimer | biographical film | ['biographical film'] | jerry bruckheimer was the producer of this biographical film.  |
| 22 | 0.714286 | Similar | PREPROCESSED | godbout v longueuil ( city of ) | longueuil | ['godbout v. longueuil', 'godbout v.', 'v. longueuil', 'the godbout v. longueuil', 'godbout v. longueuil case', 'godbout', 'longueuil'] | what court handled the godbout v. longueuil case? |
| 23 | 0.875 | Similar | PREPROCESSED | raymond a. meier | raymond | ['raymond meier', 'meier', 'was raymond meier', 'raymond'] | Which city was raymond meier born in |
| 24 | 0.714286 | Wrong Span | PREPROCESSED | afterglow | the afterglow | ['the afterglow fil', 'the afterglow fil ,', 'afterglow fil', 'afterglow fil ,', 'the afterglow'] | who did the music for the afterglow fil, |
| 25 | 0.0 | Wrong Span | PREPROCESSED | album | leo sayer | ['leo sayer'] | what is an album by leo sayer? |
| 26 | 1.0 | Similar | PREPROCESSED | drums | drum | ['drum'] | who played the drum  in the Los Angeles rock quintet Rooney |
| 27 | 0.625 | Wrong Span | PREPROCESSED | latin pop | pop music | ['latin pop music', 'pop music'] | who is an artist that creates latin pop music |
| 28 | 0.0909091 | Wrong Span | PREPROCESSED | death eaters | harry potter | ['harry potter'] | which is the name of a death eater in harry potter? |
| 29 | 0.62069 | Similar | NORMALIZED_PUNCTUATION | single - player video game | single - | ['single - player mode', 'single - player mode game', 'single - player', 'single -'] | what is a single-player mode game? |
| 30 | 0.461538 | Wrong Span | PREPROCESSED | album | compilation album | ['compilation album'] | What is a compilation album from 2006  |
| 31 | 0.047619 | Wrong Span | PREPROCESSED | studio album | arcangel | ['arcangel'] | What was a studio album recording for Arcángel |
| 32 | 0.6 | Wrong Span | PREPROCESSED | saint | the saint | ['the saint novel', 'saint novel', 'the saint'] | What type of book is the saint novel? |
| 33 | 0.5 | Wrong Span | PREPROCESSED | avila place | avila | ['avila'] | what western state does contain avila place |
| 34 | 0.0357143 | Wrong Span | PREPROCESSED | north carolina | surrey | ['surrey county', 'surrey'] | What is a city in Surrey County, north carolina? |
| 35 | 0.0 | Wrong Span | PREPROCESSED | album | johannes brahms | ['johannes brahms'] | What is an album written by Johannes Brahms |
| 36 | 0.777778 | Wrong Span | PREPROCESSED | first battle of james island | james island | ['battle of james island', 'battle of james', 'of james island', 'james island'] | Name a soldier involved in the battle of james island. |
| 37 | 0.642857 | Wrong Span | PREPROCESSED | plymouth | plymouth rock | ['plymouth rock'] | is there another attraction in plymouth other than plymouth rock |
| 38 | 0.705882 | Wrong Span | PREPROCESSED | altered beast | beast | ['beast game', 'altered beast game', 'beast'] | who is the creator of the altered beast game |
| 39 | 0.45 | Wrong Span | PREPROCESSED | the barefoot artist | barefoot | ['barefoot'] | which film created the barefoot artist  |
| 40 | 0.352941 | Wrong Span | PREPROCESSED | pornographic actor | actor | ['actor'] | who is a pornographic actor |
| 41 | 0.0 | Wrong Span | PREPROCESSED | album | century media | ['century media'] | which albums were released by the century media label? |
| 42 | 0.5 | Wrong Span | PREPROCESSED | 8833 acer | acer | ['acer'] | what is a 8833 acer |
| 43 | 1.0 | Similar | PREPROCESSED | cruisin ' | cruisin | ['cruisin'] | What release is cruisin on? |
| 44 | 0.88 | Similar | NORMALIZED_PUNCTUATION_STEM | the wonderful wizard of ha 's | the wonderful | ['the wonderful wizard of', 'the wonderful wizard of has', 'the wonderful wizard', 'wonderful wizard of', 'wonderful wizard of has', 'wonderful wizard', 'wizard of', 'the wonderful'] | What film series is the wonderful wizard of has from? |
| 45 | 0.842105 | Wrong Span, Article | PREPROCESSED | tower of london | the tower of london | ['the tower of london'] | who recorded the tower of london |
| 46 | 0.7 | Wrong Span | PREPROCESSED | outside in | outside | ['outside'] | Which genre is outside in associated with |
| 47 | 0.4 | Wrong Span | PREPROCESSED | rca | rca records | ['rca records'] | Who is an artist  signed by rca records? |
| 48 | 0.0 | Wrong Span | PREPROCESSED | action game | sega | ['sega'] | What's an action game made by sega |
| 49 | 0.647059 | Wrong Span | PREPROCESSED | film adaptation | novel | ['novel film adaptation', 'novel'] | What's an example of a novel film adaptation |


## Version 3

In version 2, the error bucketing revealed a failure to handling similar aliases's; therefore, we proceed to handle them in version 3.

In [23]:
@lru_cache(maxsize=65536)
def cached_similar_alias_normalized_punctuation_stem_to_alias(text, limit):
    cursor.execute("""SELECT set_limit(""" + str(limit) + """);
                    SELECT DISTINCT alias FROM fb_two_subject_name 
                    WHERE alias_normalized_punctuation_stem %% %s""", (text,))
    return list([r[0] for r in cursor.fetchall()])

In [25]:
from lib.table import format_pipe_table

candidates_mids = []
n_aliases = 0

for index, row in tqdm_notebook(df_answerable.iterrows(), total=df_answerable.shape[0]):
    limit = 0.8
    while limit > 0:
        is_break = False
        for i, predicted in enumerate(row['predicted_subject_names']):
            candidate_aliases = cached_alias_preprocessed_to_alias(predicted['name'])

            # Punctuation Differences
            if len(candidate_aliases) == 0:
                candidate_aliases = cached_alias_normalized_punctuation_to_alias(
                    text_normalize_punctuation(predicted['name']))

            # Suffix Differences
            if len(candidate_aliases) == 0:
                candidate_aliases = cached_alias_normalized_punctuation_stem_to_alias(
                    text_normalize_punctuation_stem(predicted['name']))

            # Other Similar Aliases
            if len(candidate_aliases) == 0:
                candidate_aliases = cached_similar_alias_normalized_punctuation_stem_to_alias(
                    text_normalize_punctuation_stem(predicted['name']), limit)

            if len(candidate_aliases) > 0:
                candidates_mids.append(cached_aliases_to_mids(candidate_aliases))
                n_aliases += len(candidate_aliases)
                is_break = True
                break
        if is_break:
            break
        limit -= 0.1
            
    if len(candidate_aliases) == 0:
        candidates_mids.append([])

evaluate_candidates(candidates_mids)
print('Average Number of Aliases:', n_aliases / len(candidates_mids))

HBox(children=(IntProgress(value=0, max=21188), HTML(value='')))

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




Answerable Precision: 0.972956 [20615 of 21188]
Answerable Recall: 1.000000 [21188 of 21188]
Expected Guessing Accuracy: 0.645286 [13942 of 21607]
Average Number of Aliases: 1.0374740419105153


#### Analysis

##### Numbers:

Version 0
- Precision: 0.964420 [10246 of 10624]
- Recall: 0.997746 [10624 of 10648]
- Expected Guessing Accuracy: 0.659801 [7025 of 10648]

Version 1
- Precision: 0.968524 [10308 of 10643]
- Recall: 0.999530 [10643 of 10648]
- Expected Guessing Accuracy: 0.664496 [7075 of 10648]
    
Verison 2
- Precision: 0.973420 [10364 of 10647]
- Recall: 0.999906 [10647 of 10648]
- Expected Guessing Accuracy: 0.669337 [7127 of 10648]
    
Version 3
- Precision: 0.974171 [10372 of 10647]
- Recall: 0.999906 [10647 of 10648]
- Expected Guessing Accuracy: 0.669558 [7129 of 10648]
- Average Number Of Aliases: 1.0379413974455296

Recall stayed the same.
Precision increased by 0.000751.

**Discussion:**

The increase here is small; therefore, it may not be worth it to include this last step in the pipeline. Without this last step, there is little room to grow otherwise with SQL queries. "Average Number of Aliases" does indicate that there is some room to grow in filtering out aliases.

## Version 4

In Version 4, we investigate alias filtering via mean candidate distance.

In [13]:
# TODO: Consider picking the alias that on average is closest to the candidate. The average summing over all
# aliases for that MID.

from Levenshtein import distance
from lib.table import format_pipe_table
import statistics

candidates_mids = []
n_aliases = 0

for index, row in tqdm_notebook(df_answerable.iterrows(), total=df_answerable.shape[0]):
    for i, predicted in enumerate(row['predicted_subject_names']):
        candidate_aliases = cached_alias_preprocessed_to_alias(predicted['name'])
        
        # Punctuation Differences
        if len(candidate_aliases) == 0:
            candidate_aliases = cached_alias_normalized_punctuation_to_alias(
                text_normalize_punctuation(predicted['name']))
            
        # Suffix Differences
        if len(candidate_aliases) == 0:
            candidate_aliases = cached_alias_normalized_punctuation_stem_to_alias(
                text_normalize_punctuation_stem(predicted['name']))
        
        # Other Similar Aliases
        if len(candidate_aliases) == 0:
            candidate_aliases = cached_similar_alias_normalized_punctuation_stem_to_alias(
                text_normalize_punctuation_stem(predicted['name']), 0.8)

        if len(candidate_aliases) > 0:
            # Filter by smallest edit distance to originally predicted name
            score = lambda a: (distance(a, predicted['name']), len(a))
            best_score = min([score(a) for a in candidate_aliases])
            candidate_aliases = [a for a in candidate_aliases if score(a) == best_score]
            
            # Copute the number of aliases
            n_aliases += len(candidate_aliases)
            mids = cached_aliases_to_mids(candidate_aliases)
            scores = []
            # IF there exists more aliases for a mid, we average
            for mid in mids:
                cursor.execute('SELECT alias FROM fb_two_name WHERE mid = %s', (mid,))
                score = statistics.mean([distance(r[0], predicted['name']) for r in cursor.fetchall()])
                scores.append(score)
            min_score = min(scores)
            mids = [mid for i, mid in enumerate(mids) if scores[i] == min_score]
            candidates_mids.append(mids)
            break
            
    if len(candidate_aliases) == 0:
        candidates_mids.append([])

evaluate_candidates(candidates_mids)
print('Average number of alaises:', n_aliases / len(candidates_mids))

HBox(children=(IntProgress(value=0, max=10648), HTML(value='')))

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




Precision: 0.831220 [8850 of 10647]
Recall: 0.999906 [10647 of 10648]
Expected Guessing Accuracy: 0.671872 [7154 of 10648]
Average number of alaises: 1.000939143501127


## Version 5

In Version 5, we investigate alias filtering via edit distance

In [ ]:
# TODO: Consider picking the alias that on average is closest to the candidate. The average summing over all
# aliases for that MID.
# TODO: Consider picking the alias with the largest amount of aggragate object mids

from Levenshtein import distance
from lib.table import format_pipe_table

candidates_mids = []
n_aliases = 0

for index, row in tqdm_notebook(df_answerable.iterrows(), total=df_answerable.shape[0]):
    for i, predicted in enumerate(row['predicted_subject_names']):
        candidate_aliases = cached_alias_preprocessed_to_alias(predicted['name'])
        
        # Punctuation Differences
        if len(candidate_aliases) == 0:
            candidate_aliases = cached_alias_normalized_punctuation_to_alias(
                text_normalize_punctuation(predicted['name']))
            
        # Suffix Differences
        if len(candidate_aliases) == 0:
            candidate_aliases = cached_alias_normalized_punctuation_stem_to_alias(
                text_normalize_punctuation_stem(predicted['name']))
        
        # Other Similar Aliases
        if len(candidate_aliases) == 0:
            candidate_aliases = cached_similar_alias_normalized_punctuation_stem_to_alias(
                text_normalize_punctuation_stem(predicted['name']), 0.8)

        if len(candidate_aliases) > 0:
            # Filter by smallest edit distance to originally predicted name
            score = lambda a: (distance(a, predicted['name']), len(a))
            best_score = min([score(a) for a in candidate_aliases])
            candidate_aliases = [a for a in candidate_aliases if score(a) == best_score]
            
            # Copute the number of aliases
            n_aliases += len(candidate_aliases)
            
            candidates_mids.append(cached_aliases_to_mids(candidate_aliases))
            break
            
    if len(candidate_aliases) == 0:
        candidates_mids.append([])

evaluate_candidates(candidates_mids)
print('Average number of alaises:', n_aliases / len(candidates_mids))

#### Analysis

##### Numbers:

Version 0
- Precision: 0.964420 [10246 of 10624]
- Recall: 0.997746 [10624 of 10648]
- Expected Guessing Accuracy: 0.659801 [7025 of 10648]

Version 1
- Precision: 0.968524 [10308 of 10643]
- Recall: 0.999530 [10643 of 10648]
- Expected Guessing Accuracy: 0.664496 [7075 of 10648]
    
Verison 2
- Precision: 0.973420 [10364 of 10647]
- Recall: 0.999906 [10647 of 10648]
- Expected Guessing Accuracy: 0.669337 [7127 of 10648]
    
Version 3
- Precision: 0.974171 [10372 of 10647]
- Recall: 0.999906 [10647 of 10648]
- Expected Guessing Accuracy: 0.669558 [7129 of 10648]
- Average Number Of Aliases: 1.0379413974455296

Version 5
- Precision: 0.972387 [10353 of 10647]
- Recall: 0.999906 [10647 of 10648]
- Expected Guessing Accuracy: 0.676108 [7199 of 10648]
- Average number of alaises: 1.000939143501127

Recall stayed the same.
Precision decreased by 0.001784.
The expected accuracy went up 0.00655 by close to half a percent.

**Discussion:**

With a small decrease in percision, we were able to reduce the number of aliases to choose from. Resulting in a 0.65% increase in our expected accuracy.

# Fin

Here we use our algorithm to generate candidates and save the results of Step 2.

In [37]:
from Levenshtein import distance
from numpy import nan

def generate_candidates(cursor, row):
    limit = 0.85
    while limit > 0:
        for i, predicted in enumerate(row['predicted_subject_names']):
            candidate_aliases = cached_alias_preprocessed_to_alias(predicted['name'])
            
            # Punctuation Differences
            if len(candidate_aliases) == 0:
                candidate_aliases = cached_alias_normalized_punctuation_to_alias(
                    text_normalize_punctuation(predicted['name']))

            # Suffix Differences
            if len(candidate_aliases) == 0:
                candidate_aliases = cached_alias_normalized_punctuation_stem_to_alias(
                    text_normalize_punctuation_stem(predicted['name']))

            # Other Similar Aliases
            if len(candidate_aliases) == 0:
                candidate_aliases = cached_similar_alias_normalized_punctuation_stem_to_alias(
                    text_normalize_punctuation_stem(predicted['name']), limit)

            if len(candidate_aliases) > 0:
                # Filter by smallest edit distance to originally predicted name
                # TODO: Look into filtering after the relation filter
                score = lambda a: (distance(a, predicted['name']), len(a))
                best_score = min([score(a) for a in candidate_aliases])
                candidate_aliases = [a for a in candidate_aliases if score(a) == best_score]
                mids = cached_aliases_to_mids(candidate_aliases)
                row['candidate_mids'] = mids
                row['predicted_start_index'] = predicted['start_index']
                row['predicted_end_index'] = predicted['end_index']
                row['predicted_subject_name'] = predicted['name']
                return row
        limit -= 0.1
        
    row['candidate_mids'] = []
    row['predicted_start_index'] = nan
    row['predicted_end_index'] = nan
    row['predicted_subject_name'] = nan
    return row

In [38]:
from functools import partial

df = df.progress_apply(partial(generate_candidates, cursor), axis=1)

HBox(children=(IntProgress(value=0, max=21687), HTML(value='')))

## Sanity

Check if `generate_candidates` works as expected

In [44]:
# Compute the accuracy just to check the implementation of `generate_candidates`.
correct = 0
expected_correct = 0
subject_name_correct = 0
for index, row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
    if row['subject'] in row['candidate_mids']:
        correct += 1
        expected_correct += 1 / len(row['candidate_mids'])
    
    if (isinstance(row['subject_name'], str) and
        text_preprocess(row['subject_name']) == row['predicted_subject_name']):
        subject_name_correct += 1
        
print('Candidate Accuracy: %f [%d of %d]' % (correct / df.shape[0], correct, df.shape[0]))
print('Expected Accuracy: %f [%d of %d]' % (expected_correct / df.shape[0], expected_correct, df.shape[0]))
# TODO: Look at subject names that are incorrect but the subject is correct
# Because that's weird.
print('Subject Name Accuracy: %f [%d of %d]' %
      (subject_name_correct / df.shape[0],subject_name_correct, df.shape[0]))

# Candidate Accuracy: 0.951169 [20628 of 21687]
# Expected Accuracy: 0.650218 [14101 of 21687]
# Subject Name Accuracy: 0.928252 [20131 of 21687]

# Candidate Accuracy: 0.953521 [20679 of 21687]
# Expected Accuracy: 0.652421 [14149 of 21687]
# Subject Name Accuracy: 0.922119 [19998 of 21687]

HBox(children=(IntProgress(value=0, max=21687), HTML(value='')))

Candidate Accuracy: 0.953521 [20679 of 21687]
Expected Accuracy: 0.652421 [14149 of 21687]
Subject Name Accuracy: 0.922119 [19998 of 21687]


## Write to Pipeline

Write step 2 results to use them in a pipeline.

In [45]:
df.to_pickle('step_2_generate_candidates.pkl')